In [1]:
! pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import glob
warnings.filterwarnings('ignore')

In [3]:
path =  "./data"
relPathApart =  f"{path}/apartment"
relPathWeather = f"{path}/apartment-weather"
year = 2016
aptNum = 114
startDate = '2016-02-01'
endDate = '2016-03-03'

In [4]:
! mkdir -p "./data"
! wget -nc "https://lass.cs.umass.edu/smarttraces/2017/apartment-electrical.tar.gz" -P "./data"
! wget -nc "https://lass.cs.umass.edu/smarttraces/2017/apartment-weather.tar.gz" -P "./data"

File ‘./data/apartment-electrical.tar.gz’ already there; not retrieving.

File ‘./data/apartment-weather.tar.gz’ already there; not retrieving.



In [5]:
! tar -xf "./data/apartment-electrical.tar.gz" -C "./data"
! tar -xf "./data/apartment-weather.tar.gz" -C "./data"

In [6]:
apartDfs = []
allFiles = glob.glob(f"{relPathApart}/{year}/*.csv")
for fileName in allFiles:
  df = pd.read_csv(fileName, parse_dates=[1], header=None, names=['date', 'power'])
  apartDfs.append(df)

print(apartDfs)

[                       date        power
0       2016-01-01 00:00:00  1.504383333
1       2016-01-01 00:01:00  1.505133333
2       2016-01-01 00:02:00  1.489983333
3       2016-01-01 00:03:00  1.487866667
4       2016-01-01 00:04:00      1.49255
...                     ...          ...
522610  2016-12-28 22:10:00       0.6538
522611  2016-12-28 22:11:00  0.654716667
522612  2016-12-28 22:12:00  0.655416667
522613  2016-12-28 22:13:00       0.6557
522614  2016-12-28 22:14:00  0.655083333

[522615 rows x 2 columns],                        date        power
0       2016-01-01 00:00:00       0.5157
1       2016-01-01 00:01:00  1.291666667
2       2016-01-01 00:02:00  2.297633333
3       2016-01-01 00:03:00  2.298883333
4       2016-01-01 00:04:00       2.2964
...                     ...          ...
503785  2016-12-15 20:25:00  9.448016667
503786  2016-12-15 20:26:00  9.449933333
503787  2016-12-15 20:27:00  9.393133333
503788  2016-12-15 20:28:00  8.602283333
503789  2016-12-15 20:29:00 

In [7]:
for i in range(len(apartDfs)):
  apartDfs[i] = apartDfs[i].loc[(apartDfs[i]['date'] > startDate) & (apartDfs[i]['date'] < endDate)]

In [ ]:
# TODO Aggregate data of all the apartment and include weather data